In [8]:
# IMPORTING REQUIRED LIBRARIES
import torch
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import dia_matrix
from typing import Any, Callable, Dict, List, Optional, Tuple, Union, cast

In [21]:
import scqubits as sc

sc.ZeroPi.supported_noise_channels()

['tphi_1_over_f_cc', 'tphi_1_over_f_flux', 't1_flux_bias_line', 't1_inductive']

In [ ]:
#global params (ZeroPi)
omega_low = 1e-9 * 2 * np.pi,  # Low frequency cutoff. Units: 2pi GHz
omega_high = 3 * 2 * np.pi,  # High frequency cutoff. Units: 2pi GHz
t_exp = 1e4  #Measurement time. Units: ns
A_cc = 1e-7,  # Critical current noise strength. Units of critical current I_c
A_flux = 1e-6,  # Flux noise strength. Units: Phi_0
M= 400  # Mutual inductance between qubit and a flux line. Units: \Phi_0 / Ampere
R_0 = 50 # Characteristic impedance of a transmission line. Units: Ohms
T = 0.015 # Typical temperature for a superconducting circuit experiment. Units: K

In [5]:
#generic functions
def process_op(energy_esys, truncated_dim):
        evectors = energy_esys[1][:, : truncated_dim]
        ##### transform operator from native basis to energy eigenbasis 
        return 


def calc_therm_ratio(
    omega: torch.Tensor
) -> torch.Tensor:
    return (sp.constants.hbar * omega) / (sp.constants.k * T)

#functions (ZeroPi)
def _cos_phi_operator(
            pt_count: int, 
            min_val: float, 
            max_value:float,
            x: float):

        vals = np.cos(np.linspace(min_val, max_value, pt_count)+x)

        cos_phi_matrix = torch.from_numpy
        (
            sparse.dia_matrix(
            (vals, [0]), shape=(pt_count, pt_count)
            ).to_array
        )
        return cos_phi_matrix

def _cos_theta_operator(ncut):
        dim_theta = 2 * ncut + 1
        cos_theta_matrix = torch.from_numpy(
            (
                0.5
                * (
                    sparse.dia_matrix(
                        ([1.0] * dim_theta, [-1]), shape=(dim_theta, dim_theta)
                    )
                    + sparse.dia_matrix(
                        ([1.0] * dim_theta, [1]), shape=(dim_theta, dim_theta)
                    )
                )
            ).toarray()
        )
        return cos_theta_matrix


def _sin_phi_operator(
        pt_count: int, #values defined for grid
        min_val: float, 
        max_value:float,
        x: float):

        vals = np.sin(np.linspace(min_val, max_value, pt_count)+x)

        sin_phi_matrix = torch.from_numpy
        (
            sparse.dia_matrix(
            (vals, [0]), shape=(pt_count, pt_count)
            ).to_array
        )
        return sin_phi_matrix
      

def _sin_theta_operator(ncut):
      
    dim_theta = 2 * ncut + 1
    sin_theta_matrix = torch.from_numpy(
            -0.5
            * 1j
            * (
                sparse.dia_matrix(
                    ([1.0] * dim_theta, [-1]), shape=(dim_theta, dim_theta)
                )
                - sparse.dia_matrix(
                    ([1.0] * dim_theta, [1]), shape=(dim_theta, dim_theta)
                )
            ).to_array()
            )
    return sin_theta_matrix
      

def d_hamiltonian_d_EJ(flux , energy_esys):
    d_potential_d_EJ_mat = -2.0 * torch.kron(
            _cos_phi_operator(x=-2.0 * np.pi * flux / 2.0),
            _cos_theta_operator())
    return process_op(native_op=d_potential_d_EJ_mat, energy_esys=energy_esys)



def d_hamiltonian_d_flux(flux, EJ, dEJ, energy_esys):
    op_1 = torch.kron(
            _sin_phi_operator(x=-2.0 * np.pi * flux / 2.0),
            _cos_theta_operator()
        )
    op_2 = torch.kron(
            _cos_phi_operator(x=-2.0 * np.pi * flux / 2.0),
            _sin_theta_operator()
        )
    d_potential_d_flux_mat =  -2.0 * np.pi * EJ * op_1 - np.pi * EJ * dEJ * op_2

    return process_op(native_op=d_potential_d_flux_mat, energy_esys=energy_esys)


def _phi_operator(
        pt_count: int, #values defined for grid
        min_val: float, 
        max_val:float):
 
    phi_matrix = sparse.dia_matrix(pt_count, pt_count)
    diag_elements = np.linspace(min_val, max_val, pt_count)
    phi_matrix.setdiag(diag_elements)

    return torch.to_numpy(phi_matrix.toarray())

def _identity_theta(ncut):
    dim_theta = 2 * ncut + 1
    return torch.identity(dim_theta)


def phi_operator(energy_esys):
    native = torch.kron(_phi_operator(),_identity_theta())
    return process_op(native_op=native, energy_esys=energy_esys)
    

In [9]:
#T1 and Tphi - from noise package 

#T1 For Some Noise Channel

def t1(
        noise_op: torch.Tensor,
        spectral_density: Callable,
        eigvals: torch.Tensor,
        eigvecs: torch.Tensor,  #row vectors
        total: bool
    ):

        # We assume that the energies in `evals` are given in the units of frequency
        # and *not* angular frequency. The function `spectral_density` is assumed to
        # take as a parameter an angular frequency, hence we have to convert.

        ground = eigvecs[:,0]
        excited = eigvecs[:,1]
        ground_E = eigvals[0]
        excited_E = eigvals[1]

        omega = 2 * np.pi * (excited_E - ground_E) * 1e9

        s = (
            spectral_density(omega) + spectral_density(-omega) ##Not sure if this is correct - ask xanthe re the comments i.         total:
            #if False return a time/rate associated with a transition from state i to state j.
            #if True return a time/rate associated with both i to j and j to i transitions
        )
        
        rate = torch.matmul(noise_op,ground.T)
        rate = torch.matmul(excited.conj(),rate)
        rate = torch.pow(torch.abs(rate) , 2) * s

        return rate

#T_phi For Some Noise Channel
def tphi(
        A_noise: float,
        noise_op: torch.Tensor,
        eigvals: torch.Tensor,
        eigvecs: torch.Tensor,
    ) -> float:

        ground = eigvecs[:,0]
        excited = eigvecs[:,1]
        ground_E = eigvals[0]
        excited_E = eigvals[1]

        rate = torch.abs(
            torch.matmul(ground.conj(),torch.matmul(noise_op,ground.T))
            - torch.matmul(excited.conj(),torch.matmul(noise_op,excited.T))
        )
        
        rate *= A_noise * np.sqrt(2 * np.abs(np.log(omega_low * t_exp)))

        # We assume that the system energies are given in units of frequency and
        # not the angular frequency, hence we have to multiply by `2\pi`
        rate *= 2 * np.pi

        return rate


#flux_bias_line_spectral_density
def spectral_density_flb(omega, R_0, T):
        therm_ratio = calc_therm_ratio(omega, T)
        s = (
            2
            * (2 * np.pi) ** 2
            * M**2
            * omega
            * sp.constants.hbar
            / complex(R_0).real
            * (1 / torch.tanh(0.5 * therm_ratio))
            / (1 + torch.exp(-therm_ratio))
            )
        
        s *= (2 * np.pi)  # We assume that system energies are given in units of frequency
        return s



#inductive_spectral_density

def q_ind_fun(omega):
    therm_ratio = abs(calc_therm_ratio(omega, T))
    therm_ratio_500MHz = calc_therm_ratio(
        torch.tensor(2 * np.pi * 500e6), T
    )
    return (
        500e6
        * (
            torch.special.scaled_modified_bessel_k0(1 / 2 * therm_ratio_500MHz)
            * torch.sinh(1 / 2 * therm_ratio_500MHz)
            / torch.exp(1 / 2 * therm_ratio_500MHz)
        )
        / (
            torch.special.scaled_modified_bessel_k0(1 / 2 * therm_ratio)
            * torch.sinh(1 / 2 * therm_ratio)
            / torch.exp(1 / 2 * therm_ratio)
        )
    )

def spectral_density_ind(omega,EL):
    therm_ratio = calc_therm_ratio(omega,T)
    s = (
        2
        * EL
        / q_ind_fun(omega)
        * (1 / torch.tanh(0.5 * torch.abs(therm_ratio)))
        / (1 + torch.exp(-therm_ratio))
    )
    s *= (
        2 * np.pi
    )  # We assume that system energies are given in units of frequency
    return s




In [ ]:
#T1 Effective (Over all noise channels)

#T2 Effective (Over all noise channels)

In [ ]:
#Create Hamiltonian For Zero Pi 


#Common
def first_derivative_matrix(prefactor):
    #NEED TO FILL IN
    return

def second_derivative_matrix(prefactor):
    #NEED TO FILL IN
    return



#Kinetic Part

def kinetic_matrix():

    identity_phi = torch.eye(pt_count)

    identity_theta = torch.eye(dim_theta)

    i_d_dphi_operator = torch.kron(first_derivative_matrix(prefactor=1j), identity_theta )

    kinetic_matrix_phi = second_derivative_matrix(prefactor =  -2.0*ECJ)

    ##
    diag_elements = 2.0 * ECS * torch.square(torch.arange(-ncut + ng, ncut + 1 + ng))
    kinetic_matrix_theta = torch.diag_embed(diag_elements, [0])  ### Need to figure out what is spare.dia_matrix is doing
    ##


    #
    diag_elements = torch.arange(-ncut, ncut + 1)
    n_theta_matrix= torch.diag_embed(diag_elements, [0])  ### Need to figure out what is spare.dia_matrix is doing
    #### native = sparse.kron(self._identity_phi(), n_theta_matrix, format="csc")
    ##### n_theta_operator = process_op(native_op=native, energy_esys=energy_esys)
    #

    kinetic_matrix = torch.kron(kinetic_matrix_phi, identity_theta)+ torch.kron(identity_phi, kinetic_matrix_theta)

    if dCJ != 0:
        kinetic_matrix -= (
                    2.0
                    * ECS
                    * dCJ
                    * i_d_dphi_operator()
                    * n_theta_operator()
        )
    return kinetic_matrix


#Potential Part 

def potential_matrix():

    grid_linspace = self.grid.make_linspace()
    phi_inductive_vals = self.EL * np.square(grid_linspace)
    phi_inductive_potential = sparse.dia_matrix(
            (phi_inductive_vals, [0]), shape=(pt_count, pt_count)
        ).tocsc()


    phi_cos_vals = np.cos(grid_linspace - 2.0 * np.pi * self.flux / 2.0)
    phi_cos_potential = sparse.dia_matrix(
            (phi_cos_vals, [0]), shape=(pt_count, pt_count)
        ).tocsc()

    phi_sin_vals = np.sin(grid_linspace - 2.0 * np.pi * self.flux / 2.0)
    phi_sin_potential = sparse.dia_matrix(
            (phi_sin_vals, [0]), shape=(pt_count, pt_count)
        ).tocsc()

    theta_cos_potential = (
            -self.EJ
            * (
                sparse.dia_matrix(
                    ([1.0] * dim_theta, [-1]), shape=(dim_theta, dim_theta)
                )
                + sparse.dia_matrix(
                    ([1.0] * dim_theta, [1]), shape=(dim_theta, dim_theta)
                )
            )
        ).tocsc()

    potential_mat = (
            sparse.kron(phi_cos_potential, theta_cos_potential, format="csc")
            + sparse.kron(phi_inductive_potential, self._identity_theta(), format="csc")
            + 2
            * self.EJ
            * sparse.kron(self._identity_phi(), self._identity_theta(), format="csc")
        )
        
        
    if dEJ != 0:
        potential_mat += (
                EJ
                * dEJ
                * sparse.kron(phi_sin_potential, self._identity_theta(), format="csc")
                * self.sin_theta_operator()
            )
    return potential_mat

In [ ]:
#Computing T2

#Pramater Values
ECS = 5
ECJ = 10
EJ=5
dCJ = 2
dEJ = 2
EL = 4
flux = 2

#Fixed Values
ng = 5
ncut = 30

#Grid Discretisation Paramaters 
pt_count = 5 #NOT SURE WHAT THIS IS? (to do with discretisation)
delta_x = 0.1 #NOT SURE WHAT THIS IS? (to do with discretisation)


#Find H




#Solve H
energy_esys = ...
eigvals = ...
eigvevcs = ...

omega = 2 * np.pi * (eigvals[1]-eigvals[0]) * 1e9

#Tphi 
tphi_1_over_f_cc = tphi(
    A_noise = A_cc, 
    noise_op= d_hamiltonian_d_EJ(flux, energy_esys), 
    eigvals=..., eigvecs=...
    )

tphi_1_over_f_flux=tphi(
    A_noise = A_flux, 
    noise_op= d_hamiltonian_d_flux(flux, EJ, dEJ, energy_esys), 
    eigvals=..., eigvecs=...
    )

#T1
t1_flux_line_basis = t1(
    noise_op=d_hamiltonian_d_flux(flux, EJ, dEJ, energy_esys), 
    spectral_density=spectral_density_flb(omega, R_0, T), 
    eigvals=..., eigvecs=...)


t1_inductive = t1(
    noise_op=phi_operator(energy_esys), 
    spectral_density=spectral_density_ind(omega,EL), 
    eigvals=..., eigvecs=...)